<a href="https://colab.research.google.com/github/rho-selynn/592-HW4/blob/main/592_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
# This mouunts your google drive to the current runtime
drive.mount('/content/mnt')
# We define a notebook path
nb_path = '/content/notebooks'
# We create a symbolic link from our drive's default "Colab Notebooks" folder to nb_path
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
# Insert nb path
sys.path.insert(0, nb_path)

Mounted at /content/mnt


Lunar Lander link: https://www.gymlibrary.ml/environments/box2d/lunar_lander/?highlight=lunar

In [2]:
#installing prerequisite display packages
!apt update && apt install xvfb python-opengl ffmpeg
#install torch and plotting packages
!pip install torchvision matplotlib seaborn pandas numpy pathlib 
#install gym and physics engine for box2d environments
!pip install gym box2d-py

#install wrapper to visualize environment
!pip install gym-notebook-wrapper
!pip install pyvirtualdisplay
import pyvirtualdisplay
disp = pyvirtualdisplay.Display()
disp.start() # Start Xvfb and set "DISPLAY" environment properly.
!pip install pfrl
import pfrl
import torch
import torch.nn
import gym
import numpy

import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import random, os.path, math, glob, csv, base64, itertools, sys
import gym
from gym.wrappers import Monitor
import gnwrapper

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

Repos from the assignment pdf

OpenAI’s gym repository https://github.com/openai/gym

PFRL’s repository https://github.com/pfnet/pfrl

PFRL Quickstart Guide: https://github.com/pfnet/pfrl/blob/master/examples/quickstart/quickstart.ipynb

Going through PFRL Quickstart Guide

In [3]:
import pfrl
import torch
import torch.nn
import gym
import numpy

In [4]:
# defining our environment
env = gym.make('CartPole-v0')
print('observation space:', env.observation_space)
print('action space:', env.action_space)

obs = env.reset()
print('initial observation:', obs)

action = env.action_space.sample()
obs, r, done, info = env.step(action)
print('next observation:', obs)
print('reward:', r)
print('done:', done)
print('info:', info)

# Uncomment to open a GUI window rendering the current state of the environment
# env.render()

observation space: Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)
action space: Discrete(2)
initial observation: [ 0.03334786 -0.00924652  0.03910794 -0.00410245]
next observation: [ 0.03316293 -0.20490688  0.03902589  0.30065848]
reward: 1.0
done: False
info: {}


In [5]:
class QFunction(torch.nn.Module):

    def __init__(self, obs_size, n_actions):
        super().__init__()
        self.l1 = torch.nn.Linear(obs_size, 50)
        self.l2 = torch.nn.Linear(50, 50)
        self.l3 = torch.nn.Linear(50, n_actions)

    def forward(self, x):
        h = x
        h = torch.nn.functional.relu(self.l1(h))
        h = torch.nn.functional.relu(self.l2(h))
        h = self.l3(h)
        return pfrl.action_value.DiscreteActionValue(h)

obs_size = env.observation_space.low.size
n_actions = env.action_space.n
q_func = QFunction(obs_size, n_actions)

In [6]:
q_func2 = torch.nn.Sequential(
    torch.nn.Linear(obs_size, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, n_actions),
    pfrl.q_functions.DiscreteActionValueHead(),
)

In [7]:
# Use Adam to optimize q_func. eps=1e-2 is for stability.
optimizer = torch.optim.Adam(q_func.parameters(), eps=1e-2)

In [8]:
# this is to create an agent and an environment

# Set the discount factor that discounts future rewards.
gamma = 0.9

# Use epsilon-greedy for exploration
explorer = pfrl.explorers.ConstantEpsilonGreedy(
    epsilon=0.3, random_action_func=env.action_space.sample)

# DQN uses Experience Replay.
# Specify a replay buffer and its capacity.
replay_buffer = pfrl.replay_buffers.ReplayBuffer(capacity=10 ** 6)

# Since observations from CartPole-v0 is numpy.float64 while
# As PyTorch only accepts numpy.float32 by default, specify
# a converter as a feature extractor function phi.
phi = lambda x: x.astype(numpy.float32, copy=False)

# Set the device id to use GPU. To use CPU only, set it to -1.
gpu = -1

# Now create an agent that will interact with the environment.
agent = pfrl.agents.DoubleDQN(
    q_func,
    optimizer,
    replay_buffer,
    gamma,
    explorer,
    replay_start_size=500,
    update_interval=1,
    target_update_interval=100,
    phi=phi,
    gpu=gpu,
)

In [9]:
n_episodes = 300
max_episode_len = 200
for i in range(1, n_episodes + 1):
    obs = env.reset()
    R = 0  # return (sum of rewards)
    t = 0  # time step
    while True:
        # Uncomment to watch the behavior in a GUI window
        # env.render()
        action = agent.act(obs)
        obs, reward, done, _ = env.step(action)
        R += reward
        t += 1
        reset = t == max_episode_len
        agent.observe(obs, reward, done, reset)
        if done or reset:
            break
    if i % 10 == 0:
        print('episode:', i, 'R:', R)
    if i % 50 == 0:
        print('statistics:', agent.get_statistics())
print('Finished.')

episode: 10 R: 15.0
episode: 20 R: 10.0
episode: 30 R: 8.0
episode: 40 R: 10.0
episode: 50 R: 12.0
statistics: [('average_q', 0.8968698), ('average_loss', 0.16107570536513913), ('cumulative_steps', 593), ('n_updates', 94), ('rlen', 593)]
episode: 60 R: 10.0
episode: 70 R: 8.0
episode: 80 R: 13.0
episode: 90 R: 11.0
episode: 100 R: 16.0
statistics: [('average_q', 5.5653386), ('average_loss', 0.2148878481797874), ('cumulative_steps', 1362), ('n_updates', 863), ('rlen', 1362)]
episode: 110 R: 50.0
episode: 120 R: 41.0
episode: 130 R: 47.0
episode: 140 R: 34.0
episode: 150 R: 29.0
statistics: [('average_q', 9.1558895), ('average_loss', 0.21501014460343867), ('cumulative_steps', 3749), ('n_updates', 3250), ('rlen', 3749)]
episode: 160 R: 183.0
episode: 170 R: 82.0
episode: 180 R: 200.0
episode: 190 R: 200.0
episode: 200 R: 173.0
statistics: [('average_q', 10.137507), ('average_loss', 0.07072342330706306), ('cumulative_steps', 11753), ('n_updates', 11254), ('rlen', 11753)]
episode: 210 R: 20

In [10]:
# CartPole-v0's max achievable return is 200
with agent.eval_mode():
    for i in range(10):
        obs = env.reset()
        R = 0
        t = 0
        while True:
            # Uncomment to watch the behavior in a GUI window
            # env.render()
            action = agent.act(obs)
            obs, r, done, _ = env.step(action)
            R += r
            t += 1
            reset = t == 200
            agent.observe(obs, r, done, reset)
            if done or reset:
                break
        print('evaluation episode:', i, 'R:', R)

evaluation episode: 0 R: 200.0
evaluation episode: 1 R: 200.0
evaluation episode: 2 R: 193.0
evaluation episode: 3 R: 197.0
evaluation episode: 4 R: 200.0
evaluation episode: 5 R: 200.0
evaluation episode: 6 R: 200.0
evaluation episode: 7 R: 200.0
evaluation episode: 8 R: 196.0
evaluation episode: 9 R: 200.0


In [11]:
# Save an agent to the 'agent' directory
agent.save('agent')

# Uncomment to load an agent from the 'agent' directory
# agent.load('agent')

In [12]:
# Here's some nify utility functions that PFRL has to make our lives easier
# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=2000,           # Train the agent for 2000 steps
    eval_n_steps=None,       # We evaluate for episodes, not time
    eval_n_episodes=10,       # 10 episodes are sampled for each evaluation
    train_max_episode_len=200,  # Maximum length of each episode
    eval_interval=1000,   # Evaluate the agent after every 1000 steps
    outdir='result',      # Save everything to 'result' directory
)

outdir:result step:84 episode:0 R:84.0
statistics:[('average_q', 9.894259), ('average_loss', 0.09188167587912176), ('cumulative_steps', 26990), ('n_updates', 26491), ('rlen', 26990)]
outdir:result step:279 episode:1 R:195.0
statistics:[('average_q', 9.934357), ('average_loss', 0.05197152412263677), ('cumulative_steps', 27185), ('n_updates', 26686), ('rlen', 27185)]
outdir:result step:479 episode:2 R:200.0
statistics:[('average_q', 9.8638315), ('average_loss', 0.051796157859498634), ('cumulative_steps', 27385), ('n_updates', 26886), ('rlen', 27385)]
outdir:result step:679 episode:3 R:200.0
statistics:[('average_q', 9.864643), ('average_loss', 0.07148183622339274), ('cumulative_steps', 27585), ('n_updates', 27086), ('rlen', 27585)]
outdir:result step:714 episode:4 R:35.0
statistics:[('average_q', 9.89705), ('average_loss', 0.06679617187706753), ('cumulative_steps', 27620), ('n_updates', 27121), ('rlen', 27620)]
outdir:result step:728 episode:5 R:14.0
statistics:[('average_q', 9.892477), 

(<pfrl.agents.double_dqn.DoubleDQN at 0x7ff3f1860090>,
 [{'average_loss': 0.07497203591337893,
   'average_q': 9.905427,
   'cumulative_steps': 27969,
   'eval_score': 200.0,
   'n_updates': 27470,
   'rlen': 27969},
  {'average_loss': 0.06914255202631465,
   'average_q': 9.9966545,
   'cumulative_steps': 28906,
   'eval_score': 200.0,
   'n_updates': 28407,
   'rlen': 28906}])